<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>
<center> <font size ="6" color='red'> Production Planning Cockpit </font></center>


In [1]:
import os
online_version = False;editmode = True
if online_version:  
    user = "muratfirat78";repo = "ProductionPlanning"
    if os.path.isdir(repo):
        !rm -rf {repo}
        
    !git clone https://github.com/{user}/{repo}.git
    %cd /content/{repo}
from DBMain import*
DataMgr.setOnlineVersion(online_version);VisMgr.setEditMode(editmode)
#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); 
tab_5 = VisMgr.getProductionProgressTab().generatePPrgTAB()
tab_3 = VisMgr.generatePLTAB(); tab_6 = VisMgr.getSchedulingTab().generatePSschTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_5,tab_3,tab_6])


tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Instance Data');tab_set.set_title(2, 'Production Progress')
tab_set.set_title(3, 'Planning');tab_set.set_title(4, 'Scheduling')
tab_set

In [21]:
#!pip install ipylab

from ipylab import JupyterFrontEnd
app = JupyterFrontEnd()
app.commands.execute('notebook:insert-cell-below')

In [ ]:
import pandas as pd
lastinput = pd.read_excel(open('Copy of Product Moves (Stock Move Line) (stock.move.line).xlsx', 'rb'))
lastinput.head(10)

nrknowns = 0


customerorders = dict()
pns = [p.getPN() for p in VisMgr.DataManager.getProducts().values()]

print("PSize of data: ",len(lastinput))

noproductnames = []
allchecked = 0
for i,r in lastinput.iterrows():
    if (r["Product"].find("kosten") != -1) or (r["Product"].find("cost") != -1):
        if not r["Product"] in noproductnames:
            noproductnames.append(r["Product"])
        continue
    allchecked+=1
    pname = r["Product"]
    if (pname.find("[") != 1) and (pname.find("]") != 1):
        pn = pname[pname.find("[")+1:pname.find("]")]
        if pn in pns:
            nrknowns+=1
            continue
    if pname in VisMgr.DataManager.getProducts():
        nrknowns+=1
    else:
        pname2 = pname[:pname.find("/")]
        if pname2 in VisMgr.DataManager.getProducts():
            nrknowns+=1
            
print("No-product names: ",noproductnames)       
print("data instances with Known products: ",nrknowns,"among all checked ones",allchecked)
print("Products in total: ",len(lastinput["Product"].unique())-len(noproductnames))

custord_df = lastinput[lastinput["To"]=="Partner Locations/Customers"]
rawmat_df = lastinput[ lastinput["From"]=="Partner Locations/Customers" ]
rawmat2_df = lastinput[lastinput["From"]=="Partner Locations/Vendors"]

ord_df = custord_df.groupby(["Product","Source"])["Quantity"].sum().reset_index()
ord_df.columns = [*ord_df.columns[:-1], 'Quantity']

for i,r in ord_df.iterrows():
    if r["Product"].find("kosten") != -1:
        continue
    customerorders[(r["Source"],r["Product"])]=r["Quantity"]
    

print("Raw Materials in data: ",len(rawmat_df))
print("Raw Materials2 in data: ",len(rawmat2_df))

print("Raw Materials unique: ",len(rawmat_df["Product"].unique()))
print("Raw Materials2 unique: ",len(rawmat2_df["Product"].unique()))

product_df = lastinput[lastinput["Product"] == "[5605-0000-0002] POM-C gzgd plaat 540x460x60"]

print("Froms ",lastinput["From"].unique())

print("Reference ",lastinput["Reference"].unique(), ', size ',len(lastinput["Reference"].unique()))

ord_df.head(50)

print("Customer orders: ",len(customerorders))


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

bom_df = pd.read_csv("Bill of Material (mrp.bom).csv")

prodname = "[00236-05-103-C] geleideblok spanrol" 
print(prodname in bom_df['Reference'].unique())

DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct(dbmgr,prodinfo):

    prodname = prodinfo
    prodno = "XXXXXX"
    if (prodinfo.find("[")> -1 ) and (prodinfo.find("]")> -1 ):
        prodno = prodinfo[prodinfo.find("[")+1:prodinfo.find("]")]
        prodname = prodinfo[prodinfo.find("]")+1:]

    if not prodname+"_["+prodno+"]" in dbmgr.getProducts():
        newprod = Product(len(dbmgr.getProducts()),prodname+"_["+prodno+"]",prodno,0)
        return newprod

    else:
        return dbmgr.getProducts()[prodname+"_["+prodno+"]"]

def MakeOperation(dbmgr,prod,oprinfo):
    
    opname = oprinfo[:oprinfo.find("(")]
    opmach = oprinfo[oprinfo.find("(")+1:oprinfo.find(")")]

    

    myres = None
    if (oprinfo.find("(")> -1 ) and (oprinfo.find(")")> -1 ):
        
        if not opmach in dbmgr.getResources():
            myres = Resource(len(dbmgr.getResources()),"Machine",opmach,16)
            dbmgr.getResources()[opmach]=myres
        else: 
            myres = dbmgr.getResources()[opmach]
    else:
        if oprinfo.find(" - ") > -1:
            opname = oprinfo[:oprinfo.find(" - ")]
            opmach = oprinfo[oprinfo.find(" - ")+3:]
            if not opmach in dbmgr.getResources():
                myres = Resource(len(dbmgr.getResources()),"Machine",opmach,16)
                dbmgr.getResources()[opmach]=myres
            else: 
                myres = dbmgr.getResources()[opmach]
            print(opname,"-",opmach)
        else:
            
            opname = oprinfo
            for rssname,myrs in dbmgr.getResources().items():
                for opr in myrs.getOperations():
                    if opr.getName()[:opr.getName().find("_")].strip() == opname:
                        myres = myrs
                        break
            if myres == None:
                if not opname in dbmgr.getResources():
                    myres = Resource(len(dbmgr.getResources()),"Machine",opname,16)
                    dbmgr.getResources()[opname]=myres
                else: 
                    myres = dbmgr.getResources()[opname]
                #print("Unrecognized:",oprinfo)
                
                
    
    optimestr = r['BoM Lines/Operations/Duration']
                    
    optime = 0
    
    if isinstance(optimestr, (int, float)):
        optime = float(1/60)*float(optimestr) # time assumed to be minute..

    newopr = Operation(len(dbmgr.getOperations()),opname+"_"+prod.getName(),optime)

    if myres !=None:
        myres.getOperations().append(newopr)
        newopr.getRequiredResources().append(myres)

    
    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()] = newopr
        
    return newopr

nopass = 0
for i,r in bom_df.iterrows():

    productrow = False
    if nopass > 0:
        nopass-=1
        continue
    
    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow= True
 
    #print("productrow ",productrow,"i",i)
  
    if productrow:
        newprod = MakeProduct(DataMgr,r['Product'])
        DataMgr.getProducts()[newprod.getName()] = newprod

        if not pd.isnull(r['BoM Lines/Component']):
            
            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'])

            quantity = r['BoM Lines/Quantity']

            newprod.getMPredecessors()[rawprod] = quantity
            newprod.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newprod)

            DataMgr.getProducts()[rawprod.getName()] = rawprod
        

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                myopr = MakeOperation(DataMgr,newprod,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')])
                newprod.getOperations().append(myopr)

               
   
            j+=1 
        nopass=len(newprod.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)
        

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))

bom_df.head(30)

DataMgr.getCustomerOrders().clear()

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))
bom_df.head(25)

In [ ]:
import pandas as pd
lastinput = pd.read_excel(open('Copy of Product Moves (Stock Move Line) (stock.move.line).xlsx', 'rb'))
lastinput.head(10)

nrknowns = 0

customerorders = dict()
pns = [p.getPN() for p in VisMgr.DataManager.getProducts().values()]

print("PSize of data: ",len(lastinput))

noproductnames = []
allchecked = 0
for i,r in lastinput.iterrows():
    if (r["Product"].find("kosten") != -1) or (r["Product"].find("cost") != -1):
        if not r["Product"] in noproductnames:
            noproductnames.append(r["Product"])
        continue
    allchecked+=1
    pname = r["Product"]
    if (pname.find("[") != 1) and (pname.find("]") != 1):
        pn = pname[pname.find("[")+1:pname.find("]")]
        if pn in pns:
            nrknowns+=1
            continue
    if pname in VisMgr.DataManager.getProducts():
        nrknowns+=1
    else:
        pname2 = pname[:pname.find("/")]
        if pname2 in VisMgr.DataManager.getProducts():
            nrknowns+=1
            
print("No-product names: ",noproductnames)       
print("data instances with Known products: ",nrknowns,"among all checked ones",allchecked)
print("Products in total: ",len(lastinput["Product"].unique())-len(noproductnames))

custord_df = lastinput[lastinput["To"]=="Partner Locations/Customers"]
rawmat_df = lastinput[ lastinput["From"]=="Partner Locations/Customers" ]
rawmat2_df = lastinput[lastinput["From"]=="Partner Locations/Vendors"]

ord_df = custord_df.groupby(["Product","Source"])["Quantity"].sum().reset_index()
ord_df.columns = [*ord_df.columns[:-1], 'Quantity']

for i,r in ord_df.iterrows():
    if r["Product"].find("kosten") != -1:
        continue
    customerorders[(r["Source"],r["Product"])]=r["Quantity"]
    

print("Raw Materials in data: ",len(rawmat_df))
print("Raw Materials2 in data: ",len(rawmat2_df))

print("Raw Materials unique: ",len(rawmat_df["Product"].unique()))
print("Raw Materials2 unique: ",len(rawmat2_df["Product"].unique()))

product_df = lastinput[lastinput["Product"] == "[5605-0000-0002] POM-C gzgd plaat 540x460x60"]

print("Froms ",lastinput["From"].unique())

print("Reference ",lastinput["Reference"].unique(), ', size ',len(lastinput["Reference"].unique()))

ord_df.head(50)

print("Customer orders: ",len(customerorders))
